In [1]:
import numpy as np
from tqdm import tqdm
from scipy.sparse import dok_matrix

def getSparse(file_):
    n, lstName, recs = 0, [], []
    with open(file_) as f:
        for line in tqdm(f):
            vals = line.strip().split('| ')
            lstName.append(vals[0])
            recs.append([])
            for val in vals[1].split(' '):
                z = val.split('^')
                recs[n].append((int(z[0]),float(z[1])))
            n += 1
    tbl, n = dok_matrix((n,300), dtype=np.float), 0
    for stat in recs:
        for x,y in stat: tbl[n,x] = y
        n += 1
    return tbl, lstName
tblSparse, lstNames = getSparse('models/books/booksX2-300_1.tems')

In [2]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier

KNN = KNeighborsClassifier(n_neighbors=11, n_jobs=6, algorithm='brute', metric='cosine')
KNN.fit(tblSparse,range(tblSparse.shape[0]))

KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='cosine',
           metric_params=None, n_jobs=6, n_neighbors=11, p=2,
           weights='uniform')

In [3]:
from multiprocessing import Pool

def doClose(i):
    app, closes = lstNames[i], []
    w,k = KNN.kneighbors(tblSparse[i], n_neighbors=11)
    n = 0
    for j in range(len(k[0])):
        try:
            ii = k[0][j]
            if i != ii:
                closes.append((lstNames[ii],w[0][j]))
                n += 1
                if n == 10:
                    break
        except:
            print(i, k[0][j], w[0][j])
    return (app,closes)

def doSave():
    global lst
    def inStr(x):
        return x[0]+'|'+str(x[1])
    ok = map(doClose,lst)
    for app,closes in ok:
        f.write(app+'\t'+'\t'.join(map(inStr,closes))+'\n')
    lst = []

In [ ]:
lst,n = [],0
#p = Pool(6)
with open('close.tsv','w') as f:
    for i in tqdm(range(len(lstNames))):
        lst.append(i)
        n += 1
        if n % 100 == 0:
            doSave()
    if len(lst) > 0:
        doSave()
#p.close()
#p.join()

In [8]:
with open('close.tsv') as f:
    for line in f:
        vals = line.strip().split('\t')
        if vals[0][:6] == 'Эрнест':
            print(vals[0].replace('_',' '))
            for val in vals[1:]:
                x = val.split('|')
                print('  ',x[0].replace('_',' '),'('+x[1]+')')

Эрнест Хемингуэй, Фиеста
   Жид Андре, Топи (0.329854024639)
   Эрнест Хемингуэй, Прощай, оружие! (0.409432438405)
   Стейнбе Джон, Райские пастбища (0.433786933512)
   Томас Манн, Непорядок и раннее горе (0.435401466025)
   Моррисон Тони, Джаз (0.447095383416)
   Эрнест Хемингуэй, Снега Килиманджаро (0.447197292918)
   Эрнест Хемингуэй, Праздник, который всегда с тобой (0.461338618287)
   Эрнест Хемингуэй, Вешние воды (0.47219363837)
   Белль Генрих, Хлеб ранних лет (0.475638525915)
   Кавабата Ясунари, Стон горы (0.486512434288)
Эрнест Хемингуэй, Праздник, который всегда с тобой
   Стейнбе Джон, Райские пастбища (0.240405438574)
   Найпол Видиадхар, Полужизнь (0.242009180487)
   Башевис Зингер Исаак, Кафетерий (0.272730040342)
   Киплинг Джозеф Редьяр, В горной Индии (0.282726741809)
   Эрнест Хемингуэй, Вешние воды (0.289251201572)
   Башевис Зингер Исаак, Друг Кафки (0.297478097602)
   Эрнест Хемингуэй, Снега Килиманджаро (0.30522238728)
   Гордимер Надин, Там ч